## Import required libraries and packages

In [1]:
import yaml
import os
from tqdm import tqdm
import time

from prompt import Prompt
from utils.LLM import LLM

In [2]:
config_file = "./config/config.yaml"
output_path = "../benchmark_dir"
agent_name = "Llaama_3_3" # Available: ChatGPT_1 (4o-mini), ChatGPT_2 (4o), Llaama_3_3, Gemma_2, Mistral
model_name = "llama-3.3-70b-versatile" # Available: gpt-4o-mini, gpt-4o, llama-3.3-70b-versatile, gemma2-9b-it, mixtral-8x7b-32768
few_shot_count = 4

## Helper functions

In [3]:
def set_agent_configuration(configuration_file_path, agent_name):
    with open(configuration_file_path, mode="r") as file:
        config = yaml.load(file, Loader=yaml.Loader)

    agent_config = config["Agent"]

    if (agent_name == "ChatGPT_1") or (agent_name == "ChatGPT_2"):
        agent_config[agent_name]["api_keys"] = [os.environ["OPENAI_API_KEY"]]
    else:
        agent_config[agent_name]["api_keys"] = [os.environ["GROQ_API_KEY"]]
    return agent_config

## Setup and Initializations

In [4]:
agent_config = set_agent_configuration(
    configuration_file_path=config_file, agent_name=agent_name
)

KeyError: 'Llaama_3_3'

## Example for Few-shot Learning

In [ ]:
prompt = Prompt(
    agent_configuration=agent_config,
    agent_name=agent_name,
    model_name=model_name,
    few_shot_count=few_shot_count
)

In [ ]:
few_shot_examples = prompt.create_few_shot_examples()

## Test set (except one-shot example)

In [ ]:
for test_index in tqdm(range(4, 111)):
    system_prompt, actual_prompt = prompt.create_actual_prompt(index=test_index)

    final_prompt = [system_prompt] + few_shot_examples.copy()
    final_prompt.append(actual_prompt)
    LLM_agent = LLM(
        agent_configuration=agent_config, agent_name=agent_name, model_name=model_name
    )

    if (agent_name == "ChatGPT_1") or (agent_name == "ChatGPT_2"): # OpenAI
        model_prediction = LLM_agent.call_chatgpt_api(final_prompt)
    else: # Groq
        model_prediction = LLM_agent.call_groq_api(final_prompt)
    model_prediction = model_prediction.split("\n")

    # Save LLM response
    LLM_agent.save_response(
        predicted_instructions=model_prediction,
        test_input_path=prompt.get_filepaths()["input"][test_index],
        few_shot_count=prompt.get_few_shot_count(),
    )

    # Save Prompt
    LLM_agent.save_prompt(
        prompt=final_prompt,
        test_input_path=prompt.get_filepaths()["input"][test_index],
        few_shot_count=prompt.get_few_shot_count(),
    )